In [1]:
# inporting our own functions
from modules.TrainTestValGen import init_gens


In [2]:
import pandas as pd
import numpy as np
import pickle
from time import sleep
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

Using TensorFlow backend.
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saber\Anaconda3\envs\te

In [3]:
# # TF memory handling 
# # TensorFlow wizardry
# config = tf.ConfigProto()
 
# # Don't pre-allocate memory; allocate as-needed
# config.gpu_options.allow_growth = True
 
# # Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# # Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Load data
BBox_df = pd.read_csv('../BBox_List_2017.csv')
de_df = pd.read_csv('../Data_Entry_2017.csv')
train_filter = pd.read_csv('../train_val_list.txt')
train_filter.columns=['Image_Index']
test_filter = pd.read_csv('../test_list.txt')
test_filter.columns=['Image_Index']

In [5]:
# clean up incorrect ages in the data
bad_ages = de_df[de_df['Patient Age'] >= 100].index

df = de_df.drop(bad_ages)

df.drop(columns=['Unnamed: 11','OriginalImagePixelSpacing[x','y]','OriginalImage[Width','Height]'], inplace=True)

df.columns = ['Image_Index', 'Finding_Labels', 'Follow-up', 'Patient ID',
       'Patient_Age', 'Patient_Gender', 'View_Position']

# Remove secondary labels
df['Finding_Labels'] = df['Finding_Labels'].apply(lambda x: x.split('|')[0])


# Join to the list of test and training to split the data set
train_df = pd.merge(train_filter,df , on='Image_Index', how='inner')
test_df = pd.merge(test_filter,df , on='Image_Index', how='inner')

print(f'train_df =  {len(train_df)}  test_df =  {len(test_df)}')

train_df =  86511  test_df =  25591


In [6]:
#Size Images 
datagen = ImageDataGenerator(rescale=.05,validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=.05)
# How we bin out the batches to run onto the GPU
BATCH_SIZE = 20




In [7]:
from _thread import start_new_thread

#Attach CSV to Images Folder and defline our traiing validation and test
def vaild_gen(batch_fit):
    global valid_generator
    valid_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        subset="validation",
        class_mode="categorical",
        target_size=(15,15),
        batch_size=batch_fit)
    
def train_gen(batch_fit):
    global train_generator
    train_generator=datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        subset="training",
        class_mode="categorical",
        target_size=(15,15),
        batch_size=batch_fit)
    
def test_gen(batch_fit):
    global test_generator
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        class_mode="categorical",
        target_size=(15,15),
        batch_size=batch_fit)
    
def init_gens(batch_fit):
    start_new_thread(vaild_gen,(batch_fit, ))
    start_new_thread(train_gen,(batch_fit ,))
    start_new_thread(test_gen,(batch_fit, ))

In [9]:
init_gens(BATCH_SIZE)

print(f' test {test_generator.batch_size} train {train_generator.batch_size} val {valid_generator.batch_size} ')

 test 20 train 20 val 20 
Found 23438 validated image filenames belonging to 15 classes.
Found 59001 validated image filenames belonging to 15 classes.Found 19666 validated image filenames belonging to 15 classes.



In [11]:
#layers
model = Sequential()
model.add(Conv2D(15, (3, 3), padding='same',
                 input_shape=(15,15,3)))
model.add(Activation('relu'))
model.add(Conv2D(15, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(15, activation='softmax'))

In [12]:
#test Shape of data
train_labels = to_categorical(train_df.shape)
train_labels.shape

(2, 86512)

In [ ]:
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


model.compile(optimizers.rmsprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    workers=16,
                    use_multiprocessing=True,
                    epochs=10)

C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'
Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\saber\Anaconda3\envs\tensorflow\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\saber\Anaconda3\envs\tensorflow\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\utils\data_utils.py", line 666, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\utils\data_utils.py", line 661, in <lambda>
    initargs=(seqs, self.random_seed))
  File "C:\Users\saber\Anac

Epoch 1/10


In [ ]:
# Resetting global Memory https://github.com/keras-team/keras/issues/12625
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [ ]:
reset_keras()